In [54]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from plotly.subplots import make_subplots
import seaborn as sns

from plottingUtils import plot_log_candlesticks
from dataUtils import dr, estimate_params

In [68]:
products = ["^GSPC","^STOXX50E"] # "^AEX","^DJI", "^IXIC", "^TYX", "GC=F", "2B7K.DE", "NVDA", "ASML", "AMD", "TSM"]#"BTC-USD", "ETH-USD", "SOL-USD",
# get historical market data
df = yf.download(products, period="1y", interval="1d")

plot_log_candlesticks(df, products)

[*********************100%%**********************]  2 of 2 completed


In [66]:
df = dr(df, products)

In [67]:
data = estimate_params(df, products)

cm_green = sns.light_palette("green", as_cmap=True)
cm_red_pos = sns.light_palette("red", as_cmap=True)
cm_red_neg = sns.light_palette("darkred", as_cmap=True, reverse=True)
data_styled = data.style.background_gradient(cmap=cm_red_pos, subset=["std"])
for green_index in ["mean", "mean-std^2/2", "mean-intrest (E)"]:
    data_styled = data_styled.background_gradient(**{'cmap': cm_green}, subset=[green_index])
for red_index in ["risk (R)", "R/E"]:
    data_styled = data_styled.background_gradient(**{'cmap': cm_red_neg}, subset=[red_index])
display(data_styled)

,mean,std,mean-std^2/2,risk (R),mean-intrest (E),R/E
^GSPC,0.001196,0.007541,0.001167,-0.002449,0.001078,-2.270963
^STOXX50E,0.000827,0.008396,0.000792,-0.002953,0.000703,-4.201362


In [ ]:
dist1 = sp.stats.t
res1 = sp.stats.fit(dist1, dr, bounds = [(0,len(dr)),(-0.1,0.1), (0,0.1)])
dist2 = sp.stats.norm
res2 = sp.stats.fit(dist2, dr, bounds = [(-0.1,0.1), (0,0.1)])
res = res1
print(res)
print(res.nllf())

fig1 = px.histogram(dr, nbins=20, marginal="box", histnorm="probability density")
fig2 = px.line(x=np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), y=dist.pdf(np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), *res.params))
fig2.update_traces(line_color='red')

fig = make_subplots(rows=2, cols=1, shared_yaxes='all', specs=[[{'type': 'box'}], [{'type': 'histogram'}]])

fig.add_trace(go.Box(fig1.data[1]), row=1, col=1)
fig.add_trace(go.Histogram(fig1.data[0]), row=2, col=1)
fig.add_trace(go.Scatter(fig2.data[0]), secondary_y=False, row=2, col=1)

fig.update_layout(yaxis=dict(domain=[0.8516, 1.0]),
                  yaxis2=dict(domain=[0.0, 0.8316],),
                  xaxis=dict(showticklabels=False),                  
                 )

fig.show()

TypeError: object of type 'function' has no len()